# Tutorial for combining votes

Whenever you generate code for your model through `arch_generate.py`, it will insert code into `core/cnn01.py` for signle model and into `core/ensemble_model.py` for ensemble version. The ensemble version is implemented by `group` to approximate independent model for better speed.

There is a example of toy3sss100scale

The single model version:

In [ ]:
class toy3sss100scale(nn.Module):
    def __init__(self, num_classes=2, act=sign, sigmoid=False, softmax=False, scale=1, bias=True):
        super(toy3sss100scale, self).__init__()
        if act == "sign":
            self.act = msign
        elif act == "signb":
            self.act = signb
        elif act == "sigmoid":
            self.act = torch.sigmoid_
        elif act == "relu":
            self.act = torch.relu_

        if softmax:
            if num_classes < 2:
                raise ValueError("num_classes expect larger than 1, but got {num_classes}")
            self.signb = softmax_
        else:
            self.signb = torch.sigmoid if sigmoid else signb

        self.conv1_si = nn.Conv2d(3, 16, kernel_size=3, padding=1, bias=bias)
        self.conv2_si = nn.Conv2d(16, 32, kernel_size=3, padding=1, bias=bias)
        self.conv3_si = nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=bias)
        self.fc4_si = nn.Linear(1024, 100, bias=bias)
        self.fc5_si = nn.Linear(100, num_classes, bias=bias)
        self.layers = ["conv1_si", "conv2_si", "conv3_si", "fc4_si", "fc5_si"]
        self.apply(_weights_init)

    def forward(self, x, input_=None, layer=None):
        # check input start from which layer
        status = -1
        for items in self.layers:
            status += 1
            if input_ is None or items in input_:
                break

        # layer 1
        if status < 1:
            if input_ != self.layers[0] + "_ap":
                out = self.conv1_si(x)
            if layer == self.layers[0] + "_projection":
                return out
            if input_ == self.layers[0] + "_ap":
                out = x
            out = msign(out) * 0.0833
            out = F.avg_pool2d(out, 2)
            if layer == self.layers[0] + "_output":
                return out

        # layer 2
        if input_ == self.layers[1]:
            out = x
        if status < 2:
            if input_ != self.layers[1] + "_ap":
                out = self.conv2_si(out)
            if layer == self.layers[1] + "_projection":
                return out
            if input_ == self.layers[1] + "_ap":
                out = x
            out = msign(out) * 0.0589
            out = F.avg_pool2d(out, 2)
            if layer == self.layers[1] + "_output":
                return out

        # layer 3
        if input_ == self.layers[2]:
            out = x
        if status < 3:
            if input_ != self.layers[2] + "_ap":
                out = self.conv3_si(out)
            if layer == self.layers[2] + "_projection":
                return out
            if input_ == self.layers[2] + "_ap":
                out = x
            out = msign(out) * 0.0417
            out = F.avg_pool2d(out, 2)
            out = out.reshape(out.size(0), -1)
            if layer == self.layers[2] + "_output":
                return out

        # layer 4
        if input_ == self.layers[3]:
            out = x
        if status < 4:
            if input_ != self.layers[3] + "_ap":
                out = self.fc4_si(out)
            if layer == self.layers[3] + "_projection":
                return out
            if input_ == self.layers[3] + "_ap":
                out = x
            out = torch.relu_(out)
            if layer == self.layers[3] + "_output":
                return out

        # layer 5
        if input_ == self.layers[4]:
            out = x
        if status < 5:
            if input_ != self.layers[4] + "_ap":
                out = self.fc5_si(out)
            if layer == self.layers[4] + "_projection":
                return out
            if input_ == self.layers[4] + "_ap":
                out = x
            out = self.signb(out)

        return out

The ensemble version:

In [ ]:
class toy3sss100scale(nn.Module):
    def __init__(self, num_classes=2, act=sign, sigmoid=False, softmax=False, scale=1, votes=1, bias=True):
        super(toy3sss100scale, self).__init__()
        self.votes = votes
        self.num_classes = num_classes
        if act == "sign":
            self.act = msign
        elif act == "signb":
            self.act = signb
        elif act == "sigmoid":
            self.act = torch.sigmoid_
        elif act == "relu":
            self.act = torch.relu_

        if softmax:
            if num_classes < 2:
                raise ValueError("num_classes expect larger than 3, but got {num_classes}")
            self.signb = softmax_
        else:
            self.signb = torch.sigmoid if sigmoid else signb

        self.conv1_si = nn.Conv2d(3, 16 * votes, kernel_size=3, padding=1, bias=bias)
        self.conv2_si = nn.Conv2d(16 * votes, 32 * votes, kernel_size=3, padding=1, bias=bias, groups=votes)
        self.conv3_si = nn.Conv2d(32 * votes, 64 * votes, kernel_size=3, padding=1, bias=bias, groups=votes)
        self.fc4_si = nn.Conv1d(votes, 100 * votes, kernel_size=1024, bias=bias, groups=votes)
        self.fc5_si = nn.Conv1d(votes, num_classes * votes, kernel_size=100, bias=bias, groups=votes)
        self.layers = ["conv1_si", "conv2_si", "conv3_si", "fc4_si", "fc5_si"]

    def forward(self, out):
        out = self.conv1_si(out)
        out = msign(out) * 0.0833
        out = F.avg_pool2d(out, 2)
        out = self.conv2_si(out)
        out = msign(out) * 0.0589
        out = F.avg_pool2d(out, 2)
        out = self.conv3_si(out)
        out = msign(out) * 0.0417
        out = F.avg_pool2d(out, 2)
        out = out.reshape((out.size(0), self.votes, -1))
        out = self.fc4_si(out)
        out = torch.relu_(out)
        out = out.reshape((out.size(0), self.votes, -1))
        out = self.fc5_si(out)
        out = out.reshape((out.size(0), self.votes, self.num_classes))
        if self.num_classes == 1:
            out = self.signb(out).squeeze(dim=-1)
            out = out.mean(dim=1).round()
        else:
            out = self.signb(out)
            out = out.mean(dim=1).argmax(dim=-1)

        return out

`combine_vote_mlp.py` for BCE, `combine_vote_new.py` for multi-class CE

python combine_vote_mlp.py --dataset cifar10 --n_classes 2 --votes 8 --no_bias 1 --scale 1 --cnn 1 --version toy3sss100scale --act sign --target cifar10_binary_toy3sss100scale_nb2_bce_bp_0 --save 

python combine_vote_new.py --dataset cifar10 --n_classes 10 --votes 8 --no_bias 1 --scale 1 --cnn 1 --version toy3sss100scale --act sign --target cifar10_toy3sss100scale_nb2_mce_bp_0 --save 
